# 분기테스트

In [3]:
# pip install -U langchain langgraph langchain-openai tiktoken
import os, json
from typing import TypedDict, List, Optional, Dict, Any

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

# ---------- 0) Config ----------
os.environ.setdefault("OPENAI_API_KEY", "PUT_YOUR_KEY_HERE")  # or set in env
MODEL_NAME = "gpt-4o-mini"  # keep it small & fast for routing

SUPERVISOR_SYSTEM_PROMPT = """
You are the “Perfume Recommendation Supervisor (Router)”. Analyze the user’s query (Korean or English) and route to exactly ONE agent below.

[Agents]
- LLM_parser         : Parses/normalizes multi-facet queries (2+ product facets).
- FAQ_agent          : Perfume knowledge / definitions / differences / general questions.
- human_fallback     : Non-perfume or off-topic queries.
- price_agent        : Price-only intents (cheapest, price, buy, discount, etc.).
- ML_agent           : Single-preference recommendations (mood/season vibe like “fresh summer”, “sweet”, etc.).

[Facets to detect (“product facets”)]
- brand            (e.g., Chanel, Dior, Creed)
- season           (spring/summer/fall/winter; “for summer/winter”)
- gender           (male/female/unisex)
- sizes            (volume in ml: 30/50/100 ml)
- day_night_score  (day/night/daily/office/club, etc.)
- concentration    (EDT/EDP/Extrait/Parfum/Cologne)

[Price intent keywords (not exhaustive)]
- Korean: 가격, 최저가, 얼마, 가격대, 구매, 판매, 할인, 어디서 사, 배송비
- English: price, cost, cheapest, buy, purchase, discount

[FAQ examples]
- Differences between EDP vs EDT, note definitions, longevity/projection, brand/line info.

[Single-preference (ML_agent) examples]
- “Recommend a cool perfume for summer”, “Recommend a sweet scent”, “One citrusy fresh pick”
  (= 0–1 of the above facets mentioned; primarily taste/mood/situation).

[Routing rules (priority)]
1) Non-perfume / off-topic → human_fallback
2) Clear price-only intent (even if one facet is present as context) → price_agent
   e.g., “Chanel No. 5 50ml cheapest price?” → price_agent
3) Count product facets in the query:
   - If facets ≥ 2 → LLM_parser
4) Otherwise (single-topic queries):
   - Perfume knowledge/definitions → FAQ_agent
   - Single taste/mood recommendation → ML_agent
5) Tie-breakers:
   - If price intent is clear → price_agent
   - If facets ≥ 2 → LLM_parser
   - Else: knowledge → FAQ_agent, taste → ML_agent

[Output format]
Return ONLY this JSON (no extra text):
{{
  "next": "<LLM_parser|FAQ_agent|human_fallback|price_agent|ML_agent>",
  "reason": "<one short English sentence>",
  "facet_count": <integer>,
  "facets": {{
    "brand": "<value or null>",
    "season": "<value or null>",
    "gender": "<value or null>",
    "sizes": "<value or null>",
    "day_night_score": "<value or null>",
    "concentration": "<value or null>"
  }},
  "scent_vibe": "<value if detected, else null>",
  "query_intent": "<price|faq|scent_pref|non_perfume|other>"
}}
""".strip()

# ---------- 1) State ----------
class AgentState(TypedDict):
    messages: List[BaseMessage]           # conversation log
    next: Optional[str]                   # routing decision key
    router_json: Optional[Dict[str, Any]] # parsed JSON from router

# ---------- 2) LLM (Supervisor) ----------
llm = ChatOpenAI(model=MODEL_NAME, temperature=0)

router_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SUPERVISOR_SYSTEM_PROMPT),
        ("user", "{query}")
    ]
)

def supervisor_node(state: AgentState) -> AgentState:
    """Call the router LLM and return parsed JSON + routing target."""
    user_query = None
    for m in reversed(state["messages"]):
        if isinstance(m, HumanMessage):
            user_query = m.content
            break
    if not user_query:
        user_query = "(empty)"

    chain = router_prompt | llm
    ai = chain.invoke({"query": user_query})
    text = ai.content

    # JSON strict parse
    chosen = "human_fallback"
    parsed: Dict[str, Any] = {}
    try:
        parsed = json.loads(text)
        maybe = parsed.get("next")
        if isinstance(maybe, str) and maybe in {"LLM_parser","FAQ_agent","human_fallback","price_agent","ML_agent"}:
            chosen = maybe
    except Exception:
        parsed = {"error": "invalid_json", "raw": text}

    msgs = state["messages"] + [AIMessage(content=text)]
    return {
        "messages": msgs,
        "next": chosen,
        "router_json": parsed
    }

# ---------- 3) Mock Agent Nodes (for testing) ----------
def passthrough(name: str):
    def _node(state: AgentState) -> AgentState:
        payload = state.get("router_json") or {}
        summary = f"[{name}] handled. reason={payload.get('reason')} facets={payload.get('facets')} intent={payload.get('query_intent')}"
        msgs = state["messages"] + [AIMessage(content=summary)]
        return {"messages": msgs, "next": None, "router_json": state.get("router_json")}
    return _node

LLM_parser      = passthrough("LLM_parser")
FAQ_agent       = passthrough("FAQ_agent")
human_fallback  = passthrough("human_fallback")
price_agent     = passthrough("price_agent")
ML_agent        = passthrough("ML_agent")

# ---------- 4) Build Graph ----------
graph = StateGraph(AgentState)

graph.add_node("supervisor", supervisor_node)
graph.add_node("LLM_parser", LLM_parser)
graph.add_node("FAQ_agent", FAQ_agent)
graph.add_node("human_fallback", human_fallback)
graph.add_node("price_agent", price_agent)
graph.add_node("ML_agent", ML_agent)

graph.set_entry_point("supervisor")

# Conditional routing
def router_edge(state: AgentState) -> str:
    return state["next"] or "human_fallback"

graph.add_conditional_edges(
    "supervisor",
    router_edge,
    {
        "LLM_parser": "LLM_parser",
        "FAQ_agent": "FAQ_agent",
        "human_fallback": "human_fallback",
        "price_agent": "price_agent",
        "ML_agent": "ML_agent",
    },
)

# End states
for node in ["LLM_parser", "FAQ_agent", "human_fallback", "price_agent", "ML_agent"]:
    graph.add_edge(node, END)

app = graph.compile()

# ---------- 5) Batch Test ----------
TEST_QUERIES = [
    "입생로랑 여성용 50ml 겨울용 향수 추천해줘.",                 
    "디올 EDP로 가을 밤(야간)에 쓸 만한 향수 있어?",                
    "EDP랑 EDT 차이가 뭐야?",                                       
    "탑노트·미들노트·베이스노트가 각각 무슨 뜻이야?",               
    "오늘 점심 뭐 먹을까?",                                         
    "오늘 서울 날씨 어때?",                                         
    "샤넬 넘버5 50ml 최저가 알려줘.",                               
    "디올 소바쥬 가격 얼마야? 어디서 사는 게 제일 싸?",             
    "여름에 시원한 향수 추천해줘.",                                 
    "달달한 향 추천해줘.",                                         
]

def run_tests():
    for q in TEST_QUERIES:
        print("="*80)
        print("Query:", q)
        init: AgentState = {
            "messages": [HumanMessage(content=q)],
            "next": None,
            "router_json": None
        }
        out = app.invoke(init)
        ai_msgs = [m for m in out["messages"] if isinstance(m, AIMessage)]
        router_raw = ai_msgs[-2].content if len(ai_msgs) >= 2 else "(no router output)"
        agent_summary = ai_msgs[-1].content if ai_msgs else "(no agent output)"
        print("Router JSON:", router_raw)
        print("Agent summary:", agent_summary)

if __name__ == "__main__":
    run_tests()


Query: 입생로랑 여성용 50ml 겨울용 향수 추천해줘.
Router JSON: {
  "next": "LLM_parser",
  "reason": "The query contains multiple product facets.",
  "facet_count": 3,
  "facets": {
    "brand": "입생로랑",
    "season": "겨울",
    "gender": "여성",
    "sizes": "50ml",
    "day_night_score": null,
    "concentration": null
  },
  "scent_vibe": null,
  "query_intent": "other"
}
Agent summary: [LLM_parser] handled. reason=The query contains multiple product facets. facets={'brand': '입생로랑', 'season': '겨울', 'gender': '여성', 'sizes': '50ml', 'day_night_score': None, 'concentration': None} intent=other
Query: 디올 EDP로 가을 밤(야간)에 쓸 만한 향수 있어?
Router JSON: {
  "next": "LLM_parser",
  "reason": "The query contains multiple facets including brand, concentration, season, and day/night score.",
  "facet_count": 4,
  "facets": {
    "brand": "Dior",
    "season": "fall",
    "gender": null,
    "sizes": null,
    "day_night_score": "night",
    "concentration": "EDP"
  },
  "scent_vibe": null,
  "query_intent": "other"
}
Ag